In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as display
%matplotlib inline

In [2]:
url = "https://raw.githubusercontent.com/emeryberger/CSrankings/gh-pages/csrankings.csv"

In [11]:
prof_names = pd.read_csv(url)

In [38]:
professor_publication_info_csv = "https://raw.githubusercontent.com/emeryberger/CSrankings/gh-pages/generated-author-info.csv"
professor_publication_info = pd.read_csv(professor_publication_info_csv)
professor_publication_info.drop('name',axis = 1,inplace = True)
professor_publication_info.drop('area',axis = 1, inplace = True)

In [16]:
undergraduate_number_cs = pd.read_csv("undergraduate_cs_research.csv")

In [49]:
top20_US_Universities = professor_publication_info[professor_publication_info['dept'].isin(['Carnegie Mellon University'
                                                                                            ,'Massachusetts Institute of Technology'
                                                                                           ,'University of California - Berkeley'
                                                                                           ,'Stanford University'
                                                                                           ,'Univ. of Illinois at Urbana-Champaign'
                                                                                           ,'Cornell University'
                                                                                           ,'University of Washington'
                                                                                           ,'Georgia Institute of Technology'
                                                                                           ,'Princeton University'
                                                                                           ,'University of Texas at Austin'
                                                                                           ,'California Institute of Technology'
                                                                                           ,'University of Michigan'
                                                                                           ,'Columbia University'
                                                                                           ,'University of California - Los Angeles'
                                                                                           ,'University of Wisconsin - Madison'
                                                                                           ,'Harvard University'
                                                                                           ,'University of California - San Diego'
                                                                                           ,'University of Maryland - College Park'
                                                                                           ,'University of Pennsylvania'
                                                                                           ,'Purdue University'
                                                                                           ,'Yale University'])]
final_dataframe = top20_US_Universities.groupby('dept').dept.count()
final_dataframe

dept
California Institute of Technology         273
Carnegie Mellon University                3557
Columbia University                       1153
Cornell University                        1704
Georgia Institute of Technology           1763
Harvard University                         644
Massachusetts Institute of Technology     2157
Princeton University                      1136
Purdue University                          883
Stanford University                       2080
Univ. of Illinois at Urbana-Champaign     1883
University of California - Berkeley       2271
University of California - Los Angeles    1013
University of California - San Diego      1443
University of Maryland - College Park     1283
University of Michigan                    1743
University of Pennsylvania                1190
University of Texas at Austin              994
University of Washington                  1638
University of Wisconsin - Madison         1125
Yale University                            487
Name: de